# Import Modul

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, render_template, request, jsonify
from urllib.parse import urlparse, parse_qs
import os
import tensorflowjs as tfjs


ModuleNotFoundError: No module named 'tensorflowjs'

# Data Loading

In [2]:
DATA_PATH = os.path.join('data', 'Dataset (2).xlsx') 
df = pd.read_excel(DATA_PATH, sheet_name='Final-BioSMA')

# Preprocessing

In [3]:
df["input_text"] = (
    df["Mata Pelajaran"] + " " +
    df["Materi"] + " " +
    df["Sub Materi"].fillna("") + " " +
    df["Jenjang"].astype(str) 
)
df["output_text"] = df['Jenjang'].astype(str) + " | " + df["Materi"] + " | " + df["Sub Materi"] + " | " + df["Link"]

In [4]:
vectorizer = TextVectorization(output_mode="tf-idf", max_tokens=10000, ngrams=(1,2))
text_ds = tf.data.Dataset.from_tensor_slices(df["input_text"]).batch(32)
vectorizer.adapt(text_ds)

X_input = vectorizer(df["input_text"])

# Modeling

In [5]:
input_dim = X_input.shape[1]
input_layer = tf.keras.Input(shape=(input_dim,))

# Encoder
x = tf.keras.layers.Dense(512, activation="relu")(input_layer)
x = tf.keras.layers.Dense(256, activation="relu")(x)
encoded = tf.keras.layers.Dense(64, activation="relu")(x) 

# Decoder
x = tf.keras.layers.Dense(256, activation="relu")(encoded)
x = tf.keras.layers.Dense(512, activation="relu")(x)
decoded = tf.keras.layers.Dense(input_dim, activation="linear")(x)



In [6]:
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [7]:
autoencoder.fit(X_input, X_input, 
                epochs=100, 
                verbose=0)

In [8]:
encoder = tf.keras.Model(inputs=input_layer, outputs=encoded)
embedded_vectors = encoder.predict(X_input)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [9]:
# Fungsi Rekomendasi
def get_recommendations(query, df, vectorizer, encoder, embedded_vectors, top_n=5):
    query_vectorized = vectorizer([query])
    query_embedded = encoder.predict(query_vectorized)
    similarities = cosine_similarity(query_embedded, embedded_vectors).flatten()
    most_similar_indices = similarities.argsort()[-top_n:][::-1]
    return df.loc[most_similar_indices, ["Jenjang","Materi", "Sub Materi", "Link", "output_text"]]

# Deploy Model

In [12]:
encoder.save('model\model.keras')

In [13]:
encoder.save('model\model.h5')

# Convert Model

In [ ]:
model=

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1866563061.py, line 1)